In [17]:
from pathlib import Path
import pandas as pd
import json
DATA_DIR = Path("../data")
PLAN_DIR = Path(DATA_DIR / "plan_demography")


In [65]:
plan_demographies = [
  *[z for z in PLAN_DIR.glob("ca10*.csv")],
  *[z for z in PLAN_DIR.glob("prec10*.csv")],
]

In [36]:
def generate_configuration(
  district_row,  
  candidate_count,
  cohesion_dict,
  alphas_dict,
):
  total_voters = district_row['HISP'] + district_row['NH_WHITE'] + district_row['NH_BLACK'] + district_row['NH_ASIAN'] + district_row['NH_OTHER']
  bloc_names = ["NH_WHITE", "NH_BLACK", "NH_ASIAN", "NH_OTHER", "HISP"]
  
  bloc_voter_prop = {
    name: district_row[name] / total_voters for name in bloc_names
  }
  cohesion_parameters = {
    name: cohesion_dict[name] for name in bloc_names
  }
  alphas = {
    name: alphas_dict[name] for name in bloc_names
  }
  slate_to_candidates = {
    name: [f"{name}_{i}" for i in range(candidate_count)] for name in bloc_names
  }
  return dict(
    bloc_voter_prop=bloc_voter_prop,
    cohesion_parameters=cohesion_parameters,
    alphas=alphas,
    slate_to_candidates=slate_to_candidates
  )
  

In [37]:
cohesion_matrix = [
  #  W      B     H    A    O
  [0.8, 0.05, 0.05, 0.05, 0.05],
  [0.7,  0.1,  0.1,  0.1, 0  ],
  [0.6,  0.1,  0.1,  0.1, 0.1],
  [0.5,  0.1,  0.1,  0.2, 0.1],
  [0.4,  0.1,  0.2,  0.2, 0.1],
]

alphas_matrix = [
  [1, 1, 1, 1, 1],
  [10,10,10,10,10],
  [0.5, 0.5, 0.5, 0.5, 0.5],
  [0.1, 0.1, 0.1, 0.1, 0.1],
]

def cohesion_matrix_to_dict(cohesion_array):
  return {
    "NH_WHITE": cohesion_array[0],
    "NH_BLACK": cohesion_array[1],
    "NH_ASIAN": cohesion_array[2],
    "NH_OTHER": cohesion_array[3],
    "HISP": cohesion_array[4],
  }

def alphas_matrix_to_dict(alphas_array):
  return {
    "NH_WHITE": alphas_array[0],
    "NH_BLACK": alphas_array[1],
    "NH_ASIAN": alphas_array[2],
    "NH_OTHER": alphas_array[3],
    "HISP": alphas_array[4],
  }
  
def generate_cohesion_parameters(cohesion_matrix):
  return {
    "NH_WHITE": cohesion_matrix_to_dict(cohesion_matrix[0]),
    "NH_BLACK": cohesion_matrix_to_dict(cohesion_matrix[1]),
    "NH_ASIAN": cohesion_matrix_to_dict(cohesion_matrix[2]),
    "NH_OTHER": cohesion_matrix_to_dict(cohesion_matrix[3]),
    "HISP": cohesion_matrix_to_dict(cohesion_matrix[4]),
  }

def generate_alphas_parameters(alphas_matrix):
  return {
    "NH_WHITE": alphas_matrix_to_dict(alphas_matrix[0]),
    "NH_BLACK": alphas_matrix_to_dict(alphas_matrix[1]),
    "NH_ASIAN": alphas_matrix_to_dict(alphas_matrix[2]),
    "NH_OTHER": alphas_matrix_to_dict(alphas_matrix[3]),
    "HISP": alphas_matrix_to_dict(alphas_matrix[4]),
  }

In [38]:
generate_cohesion_parameters([
  cohesion_matrix[0],
  cohesion_matrix[0],
  cohesion_matrix[0],
  cohesion_matrix[0],
  cohesion_matrix[0],
])

{'NH_WHITE': {'NH_WHITE': 0.8,
  'NH_BLACK': 0.05,
  'NH_ASIAN': 0.05,
  'NH_OTHER': 0.05,
  'HISP': 0.05},
 'NH_BLACK': {'NH_WHITE': 0.8,
  'NH_BLACK': 0.05,
  'NH_ASIAN': 0.05,
  'NH_OTHER': 0.05,
  'HISP': 0.05},
 'NH_ASIAN': {'NH_WHITE': 0.8,
  'NH_BLACK': 0.05,
  'NH_ASIAN': 0.05,
  'NH_OTHER': 0.05,
  'HISP': 0.05},
 'NH_OTHER': {'NH_WHITE': 0.8,
  'NH_BLACK': 0.05,
  'NH_ASIAN': 0.05,
  'NH_OTHER': 0.05,
  'HISP': 0.05},
 'HISP': {'NH_WHITE': 0.8,
  'NH_BLACK': 0.05,
  'NH_ASIAN': 0.05,
  'NH_OTHER': 0.05,
  'HISP': 0.05}}

In [39]:
generate_cohesion_parameters([
  cohesion_matrix[0],
  cohesion_matrix[0],
  cohesion_matrix[0],
  cohesion_matrix[0],
  cohesion_matrix[0],
])

{'NH_WHITE': {'NH_WHITE': 0.8,
  'NH_BLACK': 0.05,
  'NH_ASIAN': 0.05,
  'NH_OTHER': 0.05,
  'HISP': 0.05},
 'NH_BLACK': {'NH_WHITE': 0.8,
  'NH_BLACK': 0.05,
  'NH_ASIAN': 0.05,
  'NH_OTHER': 0.05,
  'HISP': 0.05},
 'NH_ASIAN': {'NH_WHITE': 0.8,
  'NH_BLACK': 0.05,
  'NH_ASIAN': 0.05,
  'NH_OTHER': 0.05,
  'HISP': 0.05},
 'NH_OTHER': {'NH_WHITE': 0.8,
  'NH_BLACK': 0.05,
  'NH_ASIAN': 0.05,
  'NH_OTHER': 0.05,
  'HISP': 0.05},
 'HISP': {'NH_WHITE': 0.8,
  'NH_BLACK': 0.05,
  'NH_ASIAN': 0.05,
  'NH_OTHER': 0.05,
  'HISP': 0.05}}

In [40]:

generate_alphas_parameters([
  alphas_matrix[0],
  alphas_matrix[0],   
  alphas_matrix[0],   
  alphas_matrix[0],   
  alphas_matrix[0],   
])

{'NH_WHITE': {'NH_WHITE': 1,
  'NH_BLACK': 1,
  'NH_ASIAN': 1,
  'NH_OTHER': 1,
  'HISP': 1},
 'NH_BLACK': {'NH_WHITE': 1,
  'NH_BLACK': 1,
  'NH_ASIAN': 1,
  'NH_OTHER': 1,
  'HISP': 1},
 'NH_ASIAN': {'NH_WHITE': 1,
  'NH_BLACK': 1,
  'NH_ASIAN': 1,
  'NH_OTHER': 1,
  'HISP': 1},
 'NH_OTHER': {'NH_WHITE': 1,
  'NH_BLACK': 1,
  'NH_ASIAN': 1,
  'NH_OTHER': 1,
  'HISP': 1},
 'HISP': {'NH_WHITE': 1,
  'NH_BLACK': 1,
  'NH_ASIAN': 1,
  'NH_OTHER': 1,
  'HISP': 1}}

In [67]:
NUM_RACES = 5

print("Running...")
for candidate_count in [3, 5]:
  for csv_path in plan_demographies:
    print(csv_path)
    df = pd.read_csv(csv_path)
    PLAN = csv_path.stem
    for i in range(len(cohesion_matrix)):
      cohesion_dict = cohesion_matrix_to_dict([cohesion_matrix[i] for x in range(0,NUM_RACES)])
      for j in range(len(alphas_matrix)):
        alphas_dict = alphas_matrix_to_dict([alphas_matrix[j] for x in range(0,NUM_RACES)])

        for idx, row in df.iterrows():
          DISTRICT = str(row['DISTRICT'])
          config = generate_configuration(row, candidate_count, cohesion_dict, alphas_dict)
          config_name = f"{PLAN}_{DISTRICT}_{candidate_count}_{"-".join(str(x) for x in cohesion_matrix[i])}_{"-".join(str(x) for x in alphas_matrix[j])}"    
          with open(DATA_DIR / "plan_configs" / f"{config_name}.json", "w") as f:
              json.dump(config, f)

Running...
../data/plan_demography/ca10-5848.csv
../data/plan_demography/ca10-1042.csv
../data/plan_demography/ca10-937.csv
../data/plan_demography/ca10-87557.csv
../data/plan_demography/ca10-5304.csv
../data/plan_demography/ca10-25218.csv
../data/plan_demography/prec10-8178.csv
../data/plan_demography/prec10-55213.csv
../data/plan_demography/prec10-86660.csv
../data/plan_demography/prec10-4287.csv
../data/plan_demography/prec10-55139.csv
../data/plan_demography/prec10-205.csv
../data/plan_demography/prec10-8698.csv
../data/plan_demography/prec10-41323.csv
../data/plan_demography/prec10-12816.csv
../data/plan_demography/ca10-5848.csv
../data/plan_demography/ca10-1042.csv
../data/plan_demography/ca10-937.csv
../data/plan_demography/ca10-87557.csv
../data/plan_demography/ca10-5304.csv
../data/plan_demography/ca10-25218.csv
../data/plan_demography/prec10-8178.csv
../data/plan_demography/prec10-55213.csv
../data/plan_demography/prec10-86660.csv
../data/plan_demography/prec10-4287.csv
../da